In [89]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
  #  for filename in filenames:
    #    print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [90]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV,KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , OneHotEncoder 
from sklearn.metrics import mean_squared_log_error

# np.sqrt(mean_squared_log_error(actual, predicted))

import xgboost as xgb

In [91]:
train_df =  pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

In [92]:
from pycaret.regression import *

In [93]:
#exp_reg101 = setup(data = train_df, target = 'time_spent', session_id=123,remove_outliers=True) 

In [94]:
#compare_models()

In [95]:
train_df.head()

,session_id,session_number,client_agent,device_details,date,purchased,added_in_cart,checked_out,time_spent
0,57f879e70d3c5fc2a98102d64c9fd84e,715,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,Desktop - Chrome,2020-01-22,1,0,0,236.886
1,a5442b0c7c33d0a811e7661e556b2de8,55,Product/8.0 iPhone/8.1.3,iPhone - iOS,2020-02-27,1,0,0,1277.455
2,305cb1486ed8610c00b37007926cb2c4,11,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like ...,iPhone - MobileWeb,2019-08-01,0,0,0,21.704
3,f2c1ecc9993f0071df91ba178450498c,2794,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Desktop - IE,2019-12-30,0,1,0,455.201
4,e460830ae295e55d2216ebdc761ab9a6,3674,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like ...,iPhone - Web,2019-09-10,0,0,0,1164.877


In [96]:
test_df.head()

,session_id,session_number,client_agent,device_details,date,purchased,added_in_cart,checked_out
0,0bf72b05b2cf70dc5e28c3c09f9c4f97,99,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,Desktop - IE,2020-01-07,1,0,0
1,6bc696d99954e4f1367e845465186aef,935,Product/160393 Android/4.3 Device/samsung_GT-I...,Android Phone - Android,2020-02-13,0,0,0
2,3d8611a5ea0f621d205b67f09f663a89,902,Product/4.2.2 iPhone/7.1.2,iPhone - iOS,2019-10-14,1,0,0
3,a0351545c032fd0da36289410c6c3354,3223,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Desktop - IE,2020-01-26,0,0,0
4,dd8999772b0ab27225fb05e5df2f1da6,231,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,Desktop - Chrome,2020-01-29,0,0,0


In [97]:
len(pd.unique(train_df['session_id']))==len(train_df)

True

In [98]:
train = train_df.copy()
test = test_df.copy()

# **BASIC EDA**

In [99]:
train = train.drop(labels=['session_id','client_agent'],axis = 1)
test = test.drop(labels=['session_id','client_agent'],axis = 1)

In [100]:
train['device_details'].value_counts()

iPhone - iOS                  1515
Desktop - Chrome               815
iPhone - Web                   665
Android Phone - Android        326
Desktop - Safari               313
Desktop - IE                   292
iPad - Web                     289
iPhone - MobileWeb             277
Desktop - Firefox              234
Android Tablet - Web           203
Other - Other                  185
Unknown - MobileWeb             87
Android Phone - Web             86
iPad - iOS                      77
Android Phone - MobileWeb       54
Android Tablet - Android         9
Android Tablet - MobileWeb       2
Name: device_details, dtype: int64

In [101]:
train.describe()

,session_number,purchased,added_in_cart,checked_out,time_spent
count,5429.000000,5429.000000,5429.000000,5429.000000,5429.000000
mean,1072.835329,0.159145,0.162092,0.017499,663.194292
std,1436.351474,0.365845,0.368570,0.131132,1713.671664
min,11.000000,0.000000,0.000000,0.000000,14.400000
25%,121.000000,0.000000,0.000000,0.000000,22.699000
50%,517.000000,0.000000,0.000000,0.000000,98.312000
75%,1397.000000,0.000000,0.000000,0.000000,600.463000
max,7722.000000,1.000000,1.000000,1.000000,38494.025000


In [102]:
train['session_number'].value_counts()

11      437
22      192
33      132
44      101
55       93
       ... 
3949      1
3696      1
5962      1
7601      1
6083      1
Name: session_number, Length: 610, dtype: int64

In [103]:
train["date"] = pd.to_datetime(train['date'])
test["date"] = pd.to_datetime(test['date'])

train["year"] = train['date'].dt.year
test["year"] = test['date'].dt.year

train["month"] = train['date'].dt.month
test["month"] = test['date'].dt.month

train['day']= train['date'].dt.day
test["day"] = test['date'].dt.day

train['diff_date'] = ((train['date'] - pd.datetime(1970,1,1)).dt.total_seconds())/(10**6)
test['diff_date'] = ((test['date'] - pd.datetime(1970,1,1)).dt.total_seconds())/(10**6)

In [104]:
train.head()

,session_number,device_details,date,purchased,added_in_cart,checked_out,time_spent,year,month,day,diff_date
0,715,Desktop - Chrome,2020-01-22,1,0,0,236.886,2020,1,22,1579.6512
1,55,iPhone - iOS,2020-02-27,1,0,0,1277.455,2020,2,27,1582.7616
2,11,iPhone - MobileWeb,2019-08-01,0,0,0,21.704,2019,8,1,1564.6176
3,2794,Desktop - IE,2019-12-30,0,1,0,455.201,2019,12,30,1577.6640
4,3674,iPhone - Web,2019-09-10,0,0,0,1164.877,2019,9,10,1568.0736


In [105]:
l = pd.unique(train["device_details"])
device_encoding = {}
x = 0
for i in l:
    if i not in device_encoding:
        device_encoding[i] = x
        x+=1
print(device_encoding)

train["device_encoding"] = train["device_details"].map(device_encoding)
test["device_encoding"] = test["device_details"].map(device_encoding)

{'Desktop - Chrome': 0, 'iPhone - iOS': 1, 'iPhone - MobileWeb': 2, 'Desktop - IE': 3, 'iPhone - Web': 4, 'Android Phone - Android': 5, 'Desktop - Firefox': 6, 'Android Tablet - Web': 7, 'iPad - Web': 8, 'Other - Other': 9, 'Desktop - Safari': 10, 'Android Phone - Web': 11, 'iPad - iOS': 12, 'Android Phone - MobileWeb': 13, 'Unknown - MobileWeb': 14, 'Android Tablet - Android': 15, 'Android Tablet - MobileWeb': 16}


In [106]:
train.head()

,session_number,device_details,date,purchased,added_in_cart,checked_out,time_spent,year,month,day,diff_date,device_encoding
0,715,Desktop - Chrome,2020-01-22,1,0,0,236.886,2020,1,22,1579.6512,0
1,55,iPhone - iOS,2020-02-27,1,0,0,1277.455,2020,2,27,1582.7616,1
2,11,iPhone - MobileWeb,2019-08-01,0,0,0,21.704,2019,8,1,1564.6176,2
3,2794,Desktop - IE,2019-12-30,0,1,0,455.201,2019,12,30,1577.6640,3
4,3674,iPhone - Web,2019-09-10,0,0,0,1164.877,2019,9,10,1568.0736,4


In [107]:
train= train[pd.notnull(train["time_spent"])]
train.head()

,session_number,device_details,date,purchased,added_in_cart,checked_out,time_spent,year,month,day,diff_date,device_encoding
0,715,Desktop - Chrome,2020-01-22,1,0,0,236.886,2020,1,22,1579.6512,0
1,55,iPhone - iOS,2020-02-27,1,0,0,1277.455,2020,2,27,1582.7616,1
2,11,iPhone - MobileWeb,2019-08-01,0,0,0,21.704,2019,8,1,1564.6176,2
3,2794,Desktop - IE,2019-12-30,0,1,0,455.201,2019,12,30,1577.6640,3
4,3674,iPhone - Web,2019-09-10,0,0,0,1164.877,2019,9,10,1568.0736,4


# **MODELLING**

In [108]:
#! pip install pycaret

In [109]:
#from pycaret.regression import *

In [110]:
#exp_reg101 = setup(data = train, target = 'time_spent', session_id=123,remove_outliers=True) 

In [111]:
#compare_models()

In [112]:
#dt = create_model('huber')

In [25]:
#print(dt)

HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)


In [26]:
#tt=tune_model(dt)

In [27]:
#final=finalize_model(tt)

In [28]:
#ans1=predict_model(final,data=test)
#ans1

In [29]:
#sub02 = pd.read_csv('Sample Submission.csv')
#sub02["time_spent"]=abs(ans1)

# COls

In [113]:
#cols = ['session_number','device_encoding','purchased','added_in_cart','checked_out','year','month','day','diff_date']
cols = ['session_number','purchased','added_in_cart','checked_out']
#cols=['session_number','device_encoding','purchased']
X = train[cols]
Y = train["time_spent"].values

# Statistics 

# Scaling

In [31]:
'''Y = train.iloc[:,6].values.reshape(-1,1)
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
X=scale.fit_transform(X)

Y=scale.fit_transform(Y)'''

'Y = train.iloc[:,6].values.reshape(-1,1)\nfrom sklearn.preprocessing import StandardScaler\nscale=StandardScaler()\nX=scale.fit_transform(X)\n\nY=scale.fit_transform(Y)'

In [32]:
#Y
import seaborn as sns 

#sns.distplot(train['purchased'],)
#sns.boxplot(train['purchased'],orient='vertical')


In [114]:
x_tr,x_tst,y_tr,y_tst = train_test_split(X,Y,test_size=0.25,random_state=0)

In [34]:
xgr = xgb.XGBRegressor(booster='gbtree',n_estimators=100)
xgr.fit(X=x_tr[cols],y=y_tr,eval_set=[(x_tr[cols],y_tr)],eval_metric="rmse",verbose=3)
ans = abs(xgr.predict(x_tst))
score = np.sqrt(mean_squared_log_error(y_tst, ans))
print(score)

[1]	validation_0-rmse:1579.62634
[2]	validation_0-rmse:1497.21509
[4]	validation_0-rmse:1393.90515
[5]	validation_0-rmse:1337.53235
[7]	validation_0-rmse:1263.91492
[8]	validation_0-rmse:1240.48413
[10]	validation_0-rmse:1208.52942
[11]	validation_0-rmse:1193.72729
[13]	validation_0-rmse:1174.22388
[14]	validation_0-rmse:1167.10913
[16]	validation_0-rmse:1155.96057
[17]	validation_0-rmse:1148.60083
[19]	validation_0-rmse:1137.93311
[20]	validation_0-rmse:1134.71387
[22]	validation_0-rmse:1128.72400
[23]	validation_0-rmse:1121.05115
[25]	validation_0-rmse:1111.64319
[26]	validation_0-rmse:1105.94983
[28]	validation_0-rmse:1097.81335
[29]	validation_0-rmse:1094.36865
[31]	validation_0-rmse:1086.96692
[32]	validation_0-rmse:1084.00317
[34]	validation_0-rmse:1074.15222
[35]	validation_0-rmse:1072.70044
[37]	validation_0-rmse:1063.58826
[38]	validation_0-rmse:1061.76404
[40]	validation_0-rmse:1057.35120
[41]	validation_0-rmse:1055.98962
[43]	validation_0-rmse:1053.20581
[44]	validation_0-rm

In [35]:
from sklearn.model_selection import GridSearchCV

para = { #when use hyperthread, xgboost may become slower
              'learning_rate': [0.01,.03, 0.05,0.1], #so called `eta` value
              'max_depth': [3,4,5],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [100]}

xgb_grid = GridSearchCV(xgr,
                        para,
                        cv = 10,
                        n_jobs = -1,
                        verbose=3)
xgb_grid.fit(x_tr[cols],y_tr)
xgb_grid.best_score_ 

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   20.1s finished


[20:04:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




0.13684032633540313

In [36]:
pred001=abs(xgb_grid.predict(x_tst))
score5=np.sqrt(mean_squared_log_error(y_tst,pred001))
score5

1.9921907208116234

In [37]:
#train.corr().time_spent

In [38]:
#feat_importances = pd.Series(xgr.feature_importances_, index=cols)
#feat_importances.nlargest(6).plot(kind='barh')
#plt.show()

In [39]:
#xgr.fit(X=X[cols],y=Y,verbose=2)
#ans = xgr.predict(test[cols])

# **FILE CREATION**

In [40]:
#sub = pd.read_csv('Sample Submission.csv')
#sub["time_spent"]=abs(ans)

In [41]:
#sub.to_csv('subm1_xgb.csv', index=False)

In [115]:
from sklearn.linear_model import Ridge

ll2=Ridge(random_state=0)
ll2.fit(x_tr,y_tr)
pred9=abs(ll2.predict(x_tst))
score10=np.sqrt(mean_squared_log_error(abs(y_tst),pred9))
score10

2.002308585604138

In [43]:
ans = ll2.predict(test[cols])

sub0 = pd.read_csv('Sample Submission.csv')
sub0["time_spent"]=abs(ans)

In [44]:
sub0.to_csv('subm0_ridge.csv', index=False)

# HUber

In [45]:
from sklearn.linear_model import HuberRegressor
reg=HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)
reg.fit(x_tr[cols],y_tr)
pred00=abs(reg.predict(x_tst))
score00=np.sqrt(mean_squared_log_error(y_tst,pred00))
score00

    

1.5028775360636681

In [47]:
'''from sklearn.ensemble import ExtraTreesRegressor as E
ee=E(random_state=0)
E.fit(,x_tr[cols],y_tr)
pred03=abs(E.predict(x_tst))
score03=np.sqrt(mean_squared_log_error(y_tst,pred03))
score03'''


'from sklearn.ensemble import ExtraTreesRegressor as E\nee=E(random_state=0)\nE.fit(,x_tr[cols],y_tr)\npred03=abs(E.predict(x_tst))\nscore03=np.sqrt(mean_squared_log_error(y_tst,pred03))\nscore03'

In [48]:
ans = reg.predict(test[cols])

sub02 = pd.read_csv('Sample Submission.csv')
sub02["time_spent"]=abs(ans)

In [49]:
sub02.to_csv('subm3_huber_final.csv', index=False)

In [50]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_tr,y_tr)
pred=abs(lr.predict(x_tst))
y_tst

array([1629.49 , 1858.31 ,   65.153, ...,   25.641,  550.943, 5522.98 ])

In [51]:
score2=np.sqrt(mean_squared_log_error(y_tst,pred))
score2

2.001676361005098

In [52]:
from sklearn.tree import DecisionTreeRegressor as D
dt=D(random_state=0)
dt.fit(x_tr[cols],y_tr)
pred2=abs(dt.predict(x_tst))
score3=np.sqrt(mean_squared_log_error(y_tst,pred2))
score3

1.951709448715558

In [53]:
x_tr.shape,

((4071, 4),)

In [54]:
from sklearn.ensemble import RandomForestRegressor as R
rr=R(n_estimators=500,random_state=0)
rr.fit(x_tr[cols],y_tr)
pred3=abs(rr.predict(x_tst))
score4=np.sqrt(mean_squared_log_error(y_tst,pred3))
score4

1.9077250442235145

In [55]:
#rr.fit(X=X[cols],y=Y)
ans = rr.predict(test[cols])


In [56]:
ans = rr.predict(test[cols])

sub2 = pd.read_csv('Sample Submission.csv')
sub2["time_spent"]=abs(ans)

In [57]:
#sub2.to_csv('subm3_Rand_final2.csv', index=False)

# SVm

In [150]:
from sklearn.svm import SVR as S
ss=S(kernel='sigmoid')
ss.fit(x_tr[cols],y_tr)
pred4=abs(ss.predict(x_tst))
score5=np.sqrt(mean_squared_log_error(y_tst,pred4))
print(score5)

1.7824638740261802


In [154]:
ans = ss.predict(test[cols])

sub003 = pd.read_csv('Sample Submission.csv')
sub003["time_spent"]=abs(ans)

In [155]:
sub003.to_csv('subm03_svm16.csv', index=False)

In [151]:
from sklearn.model_selection import GridSearchCV

para=[{'kernel':['sigmoid'],'gamma':[5,6,7,8,9,10]}]
grid=GridSearchCV(ss,param_grid=para,cv=10,n_jobs=-1)
grid.fit(x_tr[cols],y_tr)
grid.best_score_ 

-0.12136362647121077

In [153]:
pred01=abs(grid.predict(x_tst))
score5=np.sqrt(mean_squared_log_error(y_tst,pred01))
score5

1.7865169951925675

In [141]:
ans = grid.predict(test[cols])

sub004 = pd.read_csv('Sample Submission.csv')
sub004["time_spent"]=abs(ans)
sub004.to_csv('subm04_svm004.csv', index=False)

In [61]:
'''from sklearn.model_selection import GridSearchCV

para=[{'kernel':['rbf'],'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.21]}]
grid=GridSearchCV(ss,param_grid=para,cv=10,n_jobs=-1)
grid.fit(x_tr[cols],y_tr)
grid.best_score_ '''

-0.11889125616534871

In [62]:
'''pred01=abs(grid.predict(x_tst))
score5=np.sqrt(mean_squared_log_error(y_tst,pred01))
score5'''

1.7545269057858763

In [63]:
'''ans = grid.predict(test[cols])

sub001 = pd.read_csv('Sample Submission.csv')
sub001["time_spent"]=abs(ans)'''

In [64]:
sub001.to_csv('subm01_svm26.csv', index=False)

In [65]:
ans = ss.predict(test[cols])

sub3 = pd.read_csv('Sample Submission.csv')
sub3["time_spent"]=abs(ans)

In [66]:
sub3.to_csv('subm3_svm3.csv', index=False)

In [67]:
sub3.to_csv('subm3_svm001.csv', index=False)

In [68]:
from sklearn.ensemble import GradientBoostingRegressor

gbm= GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=123, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
gbm.fit(x_tr[cols],y_tr)
pred5=abs(gbm.predict(x_tst))
score6=np.sqrt(mean_squared_log_error(y_tst,pred5))
score6



1.9522265514243087

In [69]:
import lightgbm as lgb 


In [70]:
#!pip install lightgbm

In [71]:
import lightgbm as lgb 


In [72]:
ll=lgb.LGBMRegressor(n_estimators=100,random_state=0)
ll.fit(x_tr[cols],y_tr)
pred6=abs(ll.predict(x_tst))
score7=np.sqrt(mean_squared_log_error(y_tst,pred6))
score7

1.9282994010867174

In [73]:
ans = ll.predict(test[cols])

sub3 = pd.read_csv('Sample Submission.csv')
sub3["time_spent"]=abs(ans)

In [74]:
sub3.to_csv('subm3_light.csv', index=False)

In [75]:
#!pip install catboost

In [76]:
from catboost import CatBoostRegressor as cat

cc= cat(iterations=100, depth=3, learning_rate=0.1, loss_function='RMSE')
cc.fit(x_tr[cols],y_tr)
pred7=abs(cc.predict(x_tst))
score8=np.sqrt(mean_squared_log_error(y_tst,pred7))
score8 


0:	learn: 1760.4885066	total: 154ms	remaining: 15.2s
1:	learn: 1742.4830765	total: 159ms	remaining: 7.78s
2:	learn: 1726.6926638	total: 163ms	remaining: 5.28s
3:	learn: 1708.5885216	total: 168ms	remaining: 4.04s
4:	learn: 1697.2846801	total: 173ms	remaining: 3.29s
5:	learn: 1688.0295389	total: 180ms	remaining: 2.82s
6:	learn: 1680.9302851	total: 182ms	remaining: 2.42s
7:	learn: 1675.6185360	total: 184ms	remaining: 2.12s
8:	learn: 1664.9842576	total: 186ms	remaining: 1.88s
9:	learn: 1661.5247733	total: 188ms	remaining: 1.69s
10:	learn: 1656.4667037	total: 190ms	remaining: 1.54s
11:	learn: 1647.8820152	total: 193ms	remaining: 1.41s
12:	learn: 1643.6564810	total: 195ms	remaining: 1.3s
13:	learn: 1639.4717390	total: 197ms	remaining: 1.21s
14:	learn: 1634.4227559	total: 200ms	remaining: 1.13s
15:	learn: 1631.8234108	total: 203ms	remaining: 1.06s
16:	learn: 1625.7162225	total: 208ms	remaining: 1.01s
17:	learn: 1623.4970525	total: 211ms	remaining: 960ms
18:	learn: 1622.1502260	total: 213ms	re

1.9795551232313375

In [77]:
'''cc.fit(X=X[cols],y=Y)

ans = cc.predict(test[cols])

sub4 = pd.read_csv('Sample Submission.csv')
sub4["time_spent"]=abs(ans)'''

'cc.fit(X=X[cols],y=Y)\n\nans = cc.predict(test[cols])\n\nsub4 = pd.read_csv(\'Sample Submission.csv\')\nsub4["time_spent"]=abs(ans)'

In [78]:
#sub4.to_csv('subm4_cat2.csv', index=False)

In [79]:
from sklearn.ensemble import AdaBoostRegressor as gr
gg= gr(n_estimators=100,random_state=0)
gg.fit(x_tr[cols],y_tr)
pred8=abs(gg.predict(x_tst))
score9=np.sqrt(mean_squared_log_error(y_tst,pred8))
score9 




2.612471391753135

In [80]:
from sklearn.linear_model import BayesianRidge
bb= BayesianRidge()
bb.fit(x_tr[cols],y_tr)
pred10=abs(bb.predict(x_tst))
score11=np.sqrt(mean_squared_log_error(y_tst,pred10))
score11

2.3700570562725725

In [81]:
from sklearn.linear_model import ElasticNet
g= ElasticNet()
g.fit(x_tr[cols],y_tr)
pred11=abs(g.predict(x_tst))
score12=np.sqrt(mean_squared_log_error(y_tst,pred11))
score12

2.2965721525580514

In [82]:
from sklearn.linear_model import Ridge

ll2=Ridge(random_state=0)
ll2.fit(x_tr[cols],y_tr)
pred9=abs(ll2.predict(x_tst))
score10=np.sqrt(mean_squared_log_error(y_tst,pred9))
score10


2.002308585604138

In [83]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(x_tr[cols])
pca.explained_variance_ratio_

array([9.99999865e-01, 6.89739165e-08])

In [84]:
from mlxtend.regressor import StackingCVRegressor
stack= StackingCVRegressor(regressors=(ss,rr),meta_regressor=ll2)
stack.fit(x_tr[cols],y_tr)
pred12=abs(stack.predict(x_tst))
score13=np.sqrt(mean_squared_log_error(y_tst,pred12))
print(score13)



2.2108347042256153


In [85]:
'''from mlxtend.regressor import StackingCVRegressor
stack2= StackingCVRegressor(regressors=(stack,rr,dt,gg),meta_regressor=ss)
stack2.fit(x_tr[cols],y_tr)
pred13=abs(stack2.predict(x_tst))
score14=np.sqrt(mean_squared_log_error(y_tst,pred13))
score14'''

'from mlxtend.regressor import StackingCVRegressor\nstack2= StackingCVRegressor(regressors=(stack,rr,dt,gg),meta_regressor=ss)\nstack2.fit(x_tr[cols],y_tr)\npred13=abs(stack2.predict(x_tst))\nscore14=np.sqrt(mean_squared_log_error(y_tst,pred13))\nscore14'

In [86]:

ans = stack.predict(test[cols])

sub6 = pd.read_csv('Sample Submission.csv')
sub6["time_spent"]=abs(ans)

In [87]:
sub6.to_csv('subm3_stack123.csv', index=False)

In [88]:
score13=np.sqrt(mean_squared_log_error(y_tst,ans))
score13

ValueError: Found input variables with inconsistent numbers of samples: [1358, 2327]

In [160]:
! pip install torch


In [159]:
import torch

ModuleNotFoundError: No module named 'torch'